In [11]:
import ee
import geemap
from sqlalchemy import create_engine  
from urllib.parse import quote
import geopandas as gpd

ee.Initialize()

In [2]:
dataset = ee.ImageCollection('NOAA/GFS0P25')

In [3]:
dataset

In [4]:
# print the name of the bands
dataset.first().bandNames().getInfo()

['temperature_2m_above_ground',
 'specific_humidity_2m_above_ground',
 'relative_humidity_2m_above_ground',
 'u_component_of_wind_10m_above_ground',
 'v_component_of_wind_10m_above_ground',
 'total_precipitation_surface',
 'precipitable_water_entire_atmosphere',
 'total_cloud_cover_entire_atmosphere',
 'downward_shortwave_radiation_flux']

In [5]:
wind = dataset.select("u_component_of_wind_10m_above_ground")

In [ ]:
# read data from postgres database
db_connection_url = "postgresql://admin:%@localhost:5432/postgres" % quote("DBPASS")
con = create_engine(db_connection_url)  
sql = "SELECT * FROM public."

df = gpd.GeoDataFrame.from_postgis(sql, con)  

In [8]:
windVis = {
  "min": 0.0,
  "max": 60.0,
  "palette": ['black', '0dffff', '0524ff', 'ffffff'],
}

Map = geemap.Map()
Map.addLayer(wind.first(), windVis, "Wind speed")

In [9]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…